In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy.spatial import distance
import math
import os
import glob

from PIL import Image, ImageOps, ImageEnhance
from scipy import ndimage as ndi
from skimage.feature import peak_local_max
from skimage.morphology import watershed
import ast

In [2]:
def ConstrastStretching(I):
    #get the minimal pixel value
    c = np.amin(I)
    #get the maxmial pixel value
    d = np.amax(I)
    a = 0
    b = 255
    #given funcition
    O = (I-c)*((b-a)/(d-c))+a
    #transform the datatype into uint8
    O = O.astype('uint8')
    return O
# read imgs from specific directory
def read_img(path):
    # list used to store readed imgs
    img_array = []
    # glob files from path
    files = glob.glob(path)
    # sort files
    files.sort()
    for file in files:
        tmp_img = cv2.imread(file)
        img_array.append(tmp_img)
    return img_array

In [3]:
def draw_trajectory(imgs, centroids, rects):
    image_a = imgs[-1]
    for i in range(1, len(imgs)):
        last_img=imgs[i-1]
        cur_img=imgs[i]
        last_centro = centroids[i-1]
        cur_centro = centroids[i]
        last_rect = rects[i-1]
        cur_rect = rects[i]
        for k in last_centro: # pt in last pic
            dis = []
            for n in cur_centro:
                dis.append(distence(k,n))
            sorted_dis = sorted(dis)
            pt1=cur_centro[dis.index(sorted_dis[0])] # the closest pt to k
            pt2=cur_centro[dis.index(sorted_dis[1])] # the scond closest pt to k
            
            ind1=cur_centro.index(pt1)
            ind2=cur_centro.index(pt2)
            indk=last_centro.index(k)
            # area: ((up_left), (down_right))
#             ao = ((last_rect[indk][0], last_rect[indk][1]), (last_rect[indk][0]+last_rect[indk][2], last_rect[indk][1]+last_rect[indk][3]))
#             a1 = ((cur_rect[ind1][0], cur_rect[ind1][1]), (cur_rect[ind1][0]+cur_rect[ind1][2], cur_rect[ind1][1]+cur_rect[ind1][3]))
#             a2 = ((cur_rect[ind2][0], cur_rect[ind2][1]), (cur_rect[ind2][0]+cur_rect[ind2][2], cur_rect[ind2][1]+cur_rect[ind2][3]))
            diff1 = min(int(last_rect[indk][2] + cur_rect[ind1][2]), int(last_rect[indk][3] + cur_rect[ind1][3]))
            diff2 = min(int(last_rect[indk][2] + cur_rect[ind2][2]), int(last_rect[indk][3] + cur_rect[ind2][3]))
#             diff1 = distence(ao[0], a1[0])+distence(ao[1], a1[1])
#             diff2 = distence(ao[0], a2[0])+distence(ao[1], a2[1])
            dist1 = distence(k, pt1)
            dist2 = distence(k, pt2)
            diff = [diff1, diff2]
            dist = [dist1, dist2]
            ind = [ind1, ind2]
            for i in range(len(dist)):
                if dist[i]<3*min(dist)/2 and dist[i]<diff[i]:
                    
                    image_a = cv2.arrowedLine(image_a, k, cur_centro[ind[i]], (255, 255, 255), 1)
    return image_a
#  diff[i]<(last_rect[indk][2]+last_rect[indk][3]) and 
def distence(tup1, tup2):
    return abs(tup1[0]-tup2[0]) + abs(tup1[1]-tup2[1])

# pic = draw_trajectory(imgs, centro, centro_all)
def toneptwo(imgs, rects):
    pics = []
    cev = []
    for i in rects:
        temp=[]
        for j in i:
            tup = (j[0],j[1])
            temp.append(tup)
        cev.append(temp)
    for i in range(1, len(cev)):
        img_to_draw = imgs[:i+1]
        centro_to_draw = cev[:i+1]
        centroall_to_draw = rects[:i+1]
        pic = draw_trajectory(img_to_draw, centro_to_draw, centroall_to_draw)
        pics.append(pic)
        cv2.imwrite(f"zz/trajectory{i}.tif", pic)
    return pics

task 3 functions

In [4]:
from math import sqrt

def task3_puttext(image,speed=0, total_dis=0, net_dis=0, confinement_ratio=0):
    info1 = "Current speed: " + str(speed) + "   " + \
            "Total distance: " + str(total_dis)
    info2 =  "Net distance: " + str(net_dis) + "   " + \
            "Confinement ratio: " + str(confinement_ratio)
    font = cv2.FONT_HERSHEY_SIMPLEX
    info_img = cv2.putText(image, info1, (10, 20), font, 0.5, (255, 255, 255), 2)
    info_img = cv2.putText(image, info2, (10, 40), font, 0.5, (255, 255, 255), 2)
    return info_img


def user_selected_cell(image, centers, rects):
    user_select = cv2.selectROI('UserSelect', image)
    selection_center = (int(user_select[0]+user_select[2]/2), int(user_select[1]+user_select[3]/2))   
    dis_collect = []
    for c in centers:
#         M = cv2.moments(c)
#         center_x = int(M["m10"] / M["m00"])
#         center_y = int(M["m01"] / M["m00"])
        dis = abs(selection_center[0] - c[0]) + abs(selection_center[1] - c[1])
        dis_collect.append(dis)
#     cell = contours[dis_collect.index(min(dis_collect))]
    cell_center = centers[dis_collect.index(min(dis_collect))]
    cell_rect = rects[dis_collect.index(min(dis_collect))]
#     cell_boundary = cv2.boundingRect(cell)
#     cv2.rectangle(image, (cell_boundary[0], cell_boundary[1]), \
#                   (cell_boundary[0] + cell_boundary[2], cell_boundary[1] + cell_boundary[3]), (0, 0, 255), 2)
#     cv2.imshow('select', image)
#     cv2.waitKey(0)
#     cv2.destroyAllWindows()

    return cell_center, cell_rect


def task3_trajectory(last_centro, last_rect,cur_centro,cur_rect):
    dis = []
    eudis = 0
    for k in last_centro: # pt in last pic
        dis.append(distence(k,cur_centro))
    sorted_dis = sorted(dis)
    ind1 = dis.index(sorted_dis[0])
    ind2 = dis.index(sorted_dis[1])
    pt1=last_centro[ind1] # the closest pt to k
    pt2=last_centro[ind2] # the scond closest pt to k
#     print('pt1:', sorted_dis[0])
#     print('pt2:', sorted_dis[1])
#     for k in (pt1, pt2):
#         ind1=cur_centro.index(pt1)
#         ind2=cur_centro.index(pt2)
#     indk=last_centro.index(k)
    # area: ((up_left), (down_right))
#             ao = ((last_rect[indk][0], last_rect[indk][1]), (last_rect[indk][0]+last_rect[indk][2], last_rect[indk][1]+last_rect[indk][3]))
#             a1 = ((cur_rect[ind1][0], cur_rect[ind1][1]), (cur_rect[ind1][0]+cur_rect[ind1][2], cur_rect[ind1][1]+cur_rect[ind1][3]))
#             a2 = ((cur_rect[ind2][0], cur_rect[ind2][1]), (cur_rect[ind2][0]+cur_rect[ind2][2], cur_rect[ind2][1]+cur_rect[ind2][3]))
    diff1 = min(int(last_rect[ind1][2] + cur_rect[2]), int(last_rect[ind1][3] + cur_rect[3]))
    diff2 = min(int(last_rect[ind2][2] + cur_rect[2]), int(last_rect[ind1][3] + cur_rect[3]))
#             diff1 = distence(ao[0], a1[0])+distence(ao[1], a1[1])
#             diff2 = distence(ao[0], a2[0])+distence(ao[1], a2[1])
    dist1 = distence(cur_centro, pt1)
    dist2 = distence(cur_centro, pt2)
    diff = [diff1, diff2]
    dist = [dist1, dist2]
    ind = ind1 if dist1<dist2 else ind2
#         print('diff:', diff)
#         print('dist:', dist)
#         ind = [ind1, ind2]
    for i in range(len(dist)):
        if dist[i]<3*min(dist)/2 and dist[i]<diff[i]:
#                 print(1111)
            eudis = sqrt((last_centro[ind][0] - cur_centro[0])**2 + (last_centro[ind][1] - cur_centro[1])**2)
            eudis = round(eudis,2)
#                     image_a = cv2.arrowedLine(image_a, k, cur_centro[ind[i]], (255, 255, 255), 1)
#     print(eudis)    
    return eudis, last_centro[ind], last_rect[ind]

# pic = draw_trajectory(imgs, centro, centro_all)

# def cell_speed(centers, cur_cen):
#     dis_collect = []
#     for c in centers:
#         dis = sqrt((c[0] - cur_cen[0])**2 + (c[1] - cur_cen[1])**2)
# #         dis = distence(c,cur_cen)
#         dis_collect.append(dis)
#     speed = round(min(dis_collect), 2)
#     return speed

def task3_output(user_time_point, centers_s,x_y_w_h_s,selc_center, selc_rect):
    if user_time_point:
        cur_centro,cur_rect = selc_center, selc_rect
        all_eudist = []
        for i in range(user_time_point, 0, -1):
            eudist, last_centro, last_rect = task3_trajectory(centers_s[i-1], x_y_w_h_s[i-1],cur_centro,cur_rect)
            cur_centro,cur_rect = last_centro, last_rect
            all_eudist.append(eudist)
        speed = all_eudist[0]
        total_dist = round(sum(all_eudist),2)
        net_dist = round(sqrt((selc_center[0] - cur_centro[0])**2 + (selc_center[1] - cur_centro[1])**2),2)
        confinement_ratio = round(total_dist/net_dist,2)
    else:
        speed, total_dist, net_dist, confinement_ratio = 0, 0, 0, 0
    return speed, total_dist, net_dist, confinement_ratio

**********

In [5]:
# detect cell divisions in the second dataset
def mitosis_detection_2(img, thresh, contours, standard_distance):
    centers = []
    # dictionary whose keys are centers and values are corresponding radius
    radiuses = {}
    # dictionary whose keys are centers and values are corresponding contours
    contour_dic = {}
    for contour in contours:
        # get the center and radius of the contour
        (x,y), radius = cv2.minEnclosingCircle(contour)
        center = (x, y)
        centers.append(center)
        radiuses[center] = radius
        contour_dic[center] = contour
    kernel = np.ones((3,3),np.uint8)
    # further eroding the cells to divide these dividing cells into two daguther cells
    tmp = cv2.erode(thresh, kernel, iterations = 4)
    # get contours of divided cells
    _, tmp_contours, _ = cv2.findContours(tmp, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    tmp_centers = []
    for contour in tmp_contours:
        (x,y), radius = cv2.minEnclosingCircle(contour)
        center = (x, y)
        tmp_centers.append(center)
    # a set used to store the dividing centers in eroded image
    dividing_cells = set()
    # get all the euclidean distances between each centers in the eroded image.
    E_distances = distance.cdist(tmp_centers, tmp_centers, metric='euclidean')
    for x in range(len(E_distances)):
        for y in range(len(E_distances)):
            # get pairs of centers whose distance is short than standard distance
            if E_distances[x][y] < standard_distance and x != y:
                point_1 = tmp_centers[x]
                point_2 = tmp_centers[y]
                dividing_cells.add(point_1)
                dividing_cells.add(point_2)
    # used to store the dividing centers in original image
    dividing_centers = set()
    # find centers of mother cells in original image
    for center in dividing_cells:
        for x in centers:
            dist = math.sqrt((x[0]-center[0])**2 + (x[1]-center[1])**2)
            if dist < radiuses[x]:
#                 contour = contour_dic[x]
#                 cv2.drawContours(img, [contour],-1, (0,0,0), 15)
                dividing_centers.add(x)
                # circle the dividing cells with gray circle
                cv2.ellipse(img, (int(x[0]), int(x[1])), (25,25), 0, 0, 360, (125,0,0), 4)
        
    for x in centers:
        if x not in dividing_centers:
            # ellipse other cells with whrite ellipse
            cv2.ellipse(img, (int(x[0]), int(x[1])), (15,10), 0, 0, 360, (255,0,0), 2)
    return img

def Fluo(path):
    # read specific images
    img_array = read_img(path)
    # get shape of image
    (h, w, l) = img_array[0].shape
    # ready to write video with the form of XVID
    out = cv2.VideoWriter('project.avi', cv2.VideoWriter_fourcc(*'XVID'),5,(w, h))
    imgs = []
    x_y_w_h_s = []
    
    # ***** For task 3 ***************************************
    centers_s = []
    # **********************************************************
    
    for img in img_array:
        # convert image to gray image
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        # stretch the contrast of image
        img = ConstrastStretching(img)
        # utilize Gussian filter to remove part of noises
        gblur = cv2.GaussianBlur(img, (5, 5), 0)
        # binarize the given image
        ret, thresh = cv2.threshold(gblur,32,255,cv2.THRESH_BINARY)
        kernel = np.ones((4, 4),np.uint8)
        # split the cells who are closed to others
        thresh = cv2.erode(thresh, kernel, iterations = 3)
        # get the contour of binarized and eroded image
        _, contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        centers = []
        x_y_w_h = []
        for contour in contours:
            x_y_w_h.append(cv2.boundingRect(contour))
            (x,y), radius = cv2.minEnclosingCircle(contour)
            center = (x, y)
            centers.append(center)
            cv2.drawContours(img, [contour],-1, (255, 0, 0), 2)
            #cv2.ellipse(img, (int(center[0]), int(center[1])), (15,10), 0, 0, 360, (255,0,0), 5)
        x_y_w_h_s.append(x_y_w_h)
        
         # ***** For task 3 **************************************
        centers_s.append(centers)
        # *******************************************************
        #detect the cell dividing and highlight them and other normal cells with different bounding boxes
        img = mitosis_detection_2(img, thresh, contours, 30)
        
        # *******************************************************
        cell_num = "Number of cells: " + str(len(contours))
        img = cv2.putText(img, cell_num, (10, 680), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        # ***********************************************************
        
        imgs.append(img)
        #print(len(img))
        #img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        #out.write(img)
    
    # ***** For task 3 *****
    user_time_point = int(input("Which time point would you like to select a cell?\n"))
    selc_center, selc_rect = user_selected_cell(imgs[user_time_point], centers_s[user_time_point], x_y_w_h_s[user_time_point])
    #cur_centro,cur_rect = selc_center, selc_rect
    #eudis, cen,rec= task3_trajectory(centers_s[user_time_point-1], x_y_w_h_s[user_time_point-1],cur_centro,cur_rect)
    
    #eudis = cell_speed(centers_s[user_time_point-1],cell_center)
    speed, total_dist, net_dist, confinement_ratio = task3_output(user_time_point, centers_s,x_y_w_h_s,selc_center, selc_rect)
    info_img = task3_puttext(imgs[user_time_point],speed, total_dist, net_dist, confinement_ratio)
    cv2.rectangle(info_img, (selc_rect[0], selc_rect[1]),
                  (selc_rect[0] + selc_rect[2], selc_rect[1] + selc_rect[3]), (0, 0, 0), 2)
    cv2.imshow(f'Time point {user_time_point}', info_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite(f'Fluo Time point {user_time_point}.png', info_img)   
    # *************************
    
    imgs_1 = toneptwo(imgs, x_y_w_h_s)
    for img in imgs_1:
        # convert the gray image to colorful image
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        # write images into the video
        out.write(img)
    # release the videl
    out.release()

In [8]:
path = "C:/Users/Deepansh/Desktop/COMP 9517/GROUP PROJECT/images/Fluo-N2DL-HeLa/Sequence 2/*.tif"
Fluo(path)

Which time point would you like to select a cell?
5


 **********

In [ ]:
def mitosis_detection_3(img, thresh, contours, standard_distance):
    centers = []
    radiuses = {}
    contour_dic = {}
    for contour in contours:
        (x,y), radius = cv2.minEnclosingCircle(contour)
        center = (x, y)
        centers.append(center)
        radiuses[center] = radius
        contour_dic[center] = contour
    kernel = np.ones((2,2),np.uint8)
    tmp = cv2.erode(thresh, kernel, iterations = 5)
    _, tmp_contours, _ = cv2.findContours(tmp, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    tmp_centers = []
    for contour in tmp_contours:
        (x,y), radius = cv2.minEnclosingCircle(contour)
        center = (x, y)
        tmp_centers.append(center)
    dividing_cells = set()
    E_distances = distance.cdist(tmp_centers, tmp_centers, metric='euclidean')
    for x in range(len(E_distances)):
        for y in range(len(E_distances)):
            if E_distances[x][y] < standard_distance and x != y:
                point_1 = tmp_centers[x]
                point_2 = tmp_centers[y]
                dividing_cells.add(point_1)
                dividing_cells.add(point_2)
    dividing_centers = set()
    for center in dividing_cells:
        for x in centers:
            dist = math.sqrt((x[0]-center[0])**2 + (x[1]-center[1])**2)
            if dist < radiuses[x]:
                dividing_centers.add(x)
                cv2.ellipse(img, (int(x[0]), int(x[1])), (20,15), 0, 0, 360, (255,0,0), 2)
#                 contour = contour_dic[x]
#                 cv2.drawContours(img, [contour],-1, (125,0,0), 2)
    for x in centers:
        if x not in dividing_centers:
            contour = contour_dic[x]
            #cv2.drawContours(img, [contour],-1, (0,0,0), 2)
            cv2.ellipse(img, (int(x[0]), int(x[1])), (10,5), 0, 0, 360, (0,0,0), 2)
    return img

def PhC(path):
    img_array = read_img(path)
    (h, w, l) = img_array[0].shape
    out = cv2.VideoWriter('project1.avi', cv2.VideoWriter_fourcc(*'XVID'),5,(w, h))
    num = 0
    imgs = []
    x_y_w_h_s = []
    
    # ***** For task 3 *********************************
    centers_s = []
    # ***************************************************
    
    for img in img_array:
        if num > 100:
            break
        num += 1
        img2 = img.copy()
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        img = ConstrastStretching(img)
        kernel = np.ones((3,3),np.uint8)
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
        gblur = cv2.GaussianBlur(img, (5, 5), 0)
        ret, thresh = cv2.threshold(gblur,200,255,cv2.THRESH_BINARY)
        _, contours, _ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        centers = []
        x_y_w_h = []
        for contour in contours:
            x_y_w_h.append(cv2.boundingRect(contour))
            (x,y), radius = cv2.minEnclosingCircle(contour)
            center = (x, y)
            centers.append(center)
            #cv2.drawContours(img, [contour],-1, (255, 0, 0), 2)
            #cv2.ellipse(img, (int(center[0]), int(center[1])), (15,10), 0, 0, 360, (255,0,0), 5)
        x_y_w_h_s.append(x_y_w_h)
        
        # ***** For task 3 *************************************************************
        centers_s.append(centers)
        # ***************************************************************************** 
        
        img2 = mitosis_detection_3(img2, thresh, contours, 20)
        
        # **************************************************************
        cell_num = "Number of cells: " + str(len(contours))
        img2 = cv2.putText(img2, cell_num, (10, 556), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        # *************************************************
        
        imgs.append(img2)
        #img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        #out.write(img2)
        #plt.imshow(img)       
    
    # ***** For task 3 *****
    user_time_point = int(input("Which time point would you like to select a cell?\n"))
    selc_center, selc_rect = user_selected_cell(imgs[user_time_point], centers_s[user_time_point], x_y_w_h_s[user_time_point])
    #cur_centro,cur_rect = selc_center, selc_rect
    #eudis, cen,rec= task3_trajectory(centers_s[user_time_point-1], x_y_w_h_s[user_time_point-1],cur_centro,cur_rect)
    
    #eudis = cell_speed(centers_s[user_time_point-1],cell_center)
    speed, total_dist, net_dist, confinement_ratio = task3_output(user_time_point, centers_s,x_y_w_h_s,selc_center, selc_rect)
    info_img = task3_puttext(imgs[user_time_point],speed, total_dist, net_dist, confinement_ratio)
    cv2.rectangle(info_img, (selc_rect[0], selc_rect[1]),
                  (selc_rect[0] + selc_rect[2], selc_rect[1] + selc_rect[3]), (0, 0, 0), 2)
    cv2.imshow(f'Time point {user_time_point}', info_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite(f'Phc Time point {user_time_point}.png', info_img)   
    # *************************
    
    imgs_1 = toneptwo(imgs, x_y_w_h_s)
    for img in imgs_1:
        #img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
        out.write(img)
    out.release()
      

In [ ]:
path = "C:/Users/Deepansh/Desktop/COMP 9517/GROUP PROJECT/images/PhC-C2DL-PSC/Sequence 2/t%03d.tif"
PhC(path)